In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qutip import *
from qiskit import *

In [ ]:
# Constants

I = np.eye(2)
sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,-1j],[1j,0]])
sigma_z = np.array([[1,0],[0,-1]])

In [ ]:
# "Check" helper functions

def check_trace(rho):
    """
    Check if the trace of the density matrix is 1.
    """
    return np.isclose(rho.tr(), 1.0, atol=1e-10)

def check_hermitian(H):
    """
    Check if the operator is Hermitian.
    """
    return np.allclose(H, H.dag(), atol=1e-10)

def check_unitary(U):
    """
    Check if the operator is unitary.
    """
    return np.allclose(U * U.dag(), I, atol=1e-10) and np.allclose(U.dag() * U, I, atol=1e-10)

def check_psd(mat):
    """
    Check if the matrix is positive semi-definite.
    """
    return np.all(np.linalg.eigvals(mat) >= -1e-10)

def check_choi_decomposition(choi_matrix, pos_choi, neg_choi):
    """
    Check if the Choi matrix is decomposed correctly.
    """
    return np.all(choi_matrix - pos_choi + neg_choi >= -1e-10)


In [ ]:
# "Compute" helper functions

def compute_fidelity(rho1, rho2):
    """
    Calculate the fidelity between two density matrices.
    """
    return (rho1 * rho2).tr().sqrt()

def compute_expectation_value(rho, A):
    """
    Calculate the expectation value of an operator A with respect to a density matrix rho.
    """
    return (rho * A).tr()

In [ ]:
def choi_adjointchoi_transform(choi_matrix):
    """
    Convert the Choi matrix to its adjoint form, and vice versa (As defined in the supplementary material of the paper).
    """
    if isinstance(choi_matrix, Qobj):
        U_23 = Qobj(np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]]))
        adjoint_choi = choi_matrix.transform(U_23)
        adjoint_choi = adjoint_choi.conj()
    else:
        U_23 = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
        adjoint_choi = U_23 @ choi_matrix @ U_23
        adjoint_choi = adjoint_choi.conjugate()
    
    return adjoint_choi


def construct_choi_matrix(map):
    """
    Construct the Choi matrix for the map.
    """
    
    # Using the fact that E_ij can be expressed as sum of pauli matrices + Equation (17) of paper's suplementary material
    # w = np.array([[np.array([1,0,0,1]), np.array([0,1,1j,0])], [np.array([0,1,-1j,0]), np.array([1,0,0,-1])]])

    # choi_matrix = np.zeros((4,4), dtype=complex)
    # for i in range(2):
    #     for j in range(2):
    #         Tw = map @ w[i,j]
    #         eval_Eij = Tw[0]*I + Tw[1]*sigma_x + Tw[2]*sigma_y + Tw[3]*sigma_z
    #         choi_matrix[2*i:2*i+2,2*j:2*j+2] = eval_Eij/2
    choi_matrix = to_choi(map)

    return choi_matrix


def positive_and_negative_eigenspaces(choi_matrix):
    """
    Compute the positive and negative eigenspaces of the Choi matrix.
    """
    # eigenvalues, eigenvectors = np.linalg.eig(choi_matrix)
    eigenvalues = choi_matrix.eigenstates()[0]
    eigenvectors = choi_matrix.eigenstates()[1]
    
    # Positive eigenspace
    pos_eigenvalues = eigenvalues[eigenvalues > 0]
    pos_eigenvectors = eigenvectors[eigenvalues > 0]
    
    # Negative eigenspace
    neg_eigenvalues = eigenvalues[eigenvalues < 0]
    neg_eigenvectors = eigenvectors[eigenvalues < 0]
    
    return pos_eigenvalues, pos_eigenvectors, neg_eigenvalues, neg_eigenvectors


def construct_CP_map_from_eigenspace(eigenvalues, eigenvectors):
    """
    Construct the map from the positive and negative eigenspaces.
    """
    # dim = eigenvectors.shape[0]
    # CP_map = np.zeros((dim, dim), dtype=complex)
    eig_list = []
    for i in range(len(eigenvalues)):
        eig_list.append(eigenvalues[i] * (eigenvectors[i]*eigenvectors[i].dag()))

    CP_map = sum(eig_list)
    
    return CP_map


def decompose_into_extremal_CP_maps(choi_matrix_adjoint):

    # choi_matrix_cp_map_adjoint = choi_matrix_cp_map.T

    A = choi_matrix_adjoint[:2, :2]
    C = choi_matrix_adjoint[:2, 2:]
    C_dag = choi_matrix_adjoint[2:, :2]
    B = choi_matrix_adjoint[2:, 2:]

    # assert C.conjugate().T == C_dag, "C and C_dag are not conjugate transpose"
    # assert I - A  == B, "B is not equal to I - A"
    
    R = np.linalg.inv(np.sqrt(A)) @ C @ np.linalg.inv(np.sqrt(B))

    V, S, W_dag = np.linalg.svd(R)
    S = np.diag(S)

    theta1 = np.arccos(S[0, 0])
    theta2 = np.arccos(S[1, 1])

    U1_diag = np.diag([np.exp(1j * theta1), np.exp(1j * theta2)])
    U2_diag = np.diag([np.exp(-1j * theta1), np.exp(-1j * theta2)])

    U1 = (V @ U1_diag @ W_dag)
    U2 = (V @ U2_diag @ W_dag)

    choi1 = np.zeros((4, 4), dtype=complex)
    choi2 = np.zeros((4, 4), dtype=complex)

    choi1[:2, :2] = A
    choi1[:2, 2:] = np.sqrt(A) @ U1 @ np.sqrt(B)
    choi1[2:, :2] = np.sqrt(B) @ U1.conjugate().T @ np.sqrt(A)
    choi1[2:, 2:] =  B

    choi2[:2, :2] = A
    choi2[:2, 2:] = np.sqrt(A) @ U2 @ np.sqrt(B)
    choi2[2:, :2] = np.sqrt(B) @ U2.conjugate().T @ np.sqrt(A)
    choi2[2:, 2:] = B
    
    return choi1, choi2


def choi_to_map_in_kraus_rep(choi_extremal_adjoint):
    """
    Convert a Choi matrix to Kraus operators.
    """
    choi_extremal = choi_adjointchoi_transform(choi_extremal_adjoint)
    dim = int(np.sqrt(choi_extremal.shape[0]))
    eigenvalues, eigenvectors = np.linalg.eig(choi_extremal)
    kraus_operators = []

    for i, eigenvalue in enumerate(eigenvalues):
        if eigenvalue > 1e-10:  # Consider only positive eigenvalues
            vector = eigenvectors[:, i]
            psi = vector.reshape((dim, dim), order="F")
            K = np.sqrt(eigenvalue) * psi
            kraus_operators.append(K)
    
    return kraus_operators

def choi_to_superoperator(choi):
    """
    Convert the Choi rep to a superoperator.
    """
    kraus_operators = choi_to_map_in_kraus_rep(choi)
    dim = kraus_operators[0].shape[0]
    superoperator = np.zeros((dim**2, dim**2), dtype=complex)

    for K in kraus_operators:
        superoperator += np.kron(K.conjugate(), K)
    
    return superoperator


In [ ]:
# Lindbladian to General Dynamical Map
def get_superoperator_from_map(H, l_ops, dt):
    """
    Get the superoperator from the Hamiltonian and Lindblad operators.
    """
    # Construct the Liouvillian 
    L = liouvillian(H, l_ops)
    # Time evolution superoperator
    U = (L * dt).expm()
    
    return U



# def get_liouvillian_superoperator(H, l_ops):

#     # TODO: verify if this is the correct way to construct the Liouvillian superoperator, currently generated by copilot 


#     """
#     Get the Liouvillian superoperator from the Hamiltonian and Lindblad operators.
#     """
#     # Construct the Liouvillian superoperator
#     L = -1j * (tensor(H, qeye(2)) - tensor(qeye(2), H)).tr()
    
#     for l_op in l_ops:
#         L += (tensor(l_op.dag(), qeye(2)) @ tensor(qeye(2), l_op) - 0.5 * (tensor(l_op.dag() * l_op, qeye(2)) + tensor(qeye(2), l_op.dag() * l_op))).tr()
    
#     return L


# def get_dynamical_map(L, dt):
#     """
#     Get the dynamical map from the Liouvillian superoperator for time dt.
#     """
#     # Time evolution operator
#     U = (-1j * L * dt).expm()  
    
#     return U



In [ ]:
gamma = 1
beta = 1
omega = 1
times = np.linspace(0,1,21)
H = omega*sigmaz()/2
l_ops = []
l_ops.append(np.sqrt(gamma*np.exp(beta*omega))*sigmam())
l_ops.append(np.sqrt(gamma)*sigmap())

for t_idx in range(1,2):
    L_superoperator = get_superoperator_from_map(H, l_ops, times[t_idx])
    L_choi = construct_choi_matrix(L_superoperator)
    eigenspace_pm = positive_and_negative_eigenspaces(L_choi)

    # Check if the Choi matrix is positive semi-definite. If postive, then the map is CP and can be decomposed into extremal maps.
    if eigenspace_pm[3].size == 0:
        choi_adjoint = choi_adjointchoi_transform(L_choi)
        [choi_extremal_1, choi_extremal_2] = decompose_into_extremal_CP_maps(choi_adjoint) 
        L_extremal_super = choi_to_superoperator(choi_extremal_1)
        continue
        # U, V, nu, mu = get_unitary_angles_from_choi(choi_extremal_1)
    else:
        choi_plus = construct_CP_map_from_eigenspace(eigenspace_pm[0], eigenspace_pm[1])
        choi_minus = construct_CP_map_from_eigenspace(eigenspace_pm[2], eigenspace_pm[3])
        choi_plus_adjoint = choi_adjointchoi_transform(choi_plus)
        choi_minus_adjoint = choi_adjointchoi_transform(choi_minus)
        pass